## Nous allons créer un Decision Tree permettant de classifier differents elements, en l'occurence des fruits en fonctions de trois attributs.

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd

In [2]:
training_data = pd.read_csv("data.csv")

### Visualisez les donnees (valeurs reelles) ###

In [3]:
print(training_data)

     Color  Size  Season   Label
0      Red     1  Summer   Grape
1    Green     3  Spring   Apple
2      Red     3  Winter  Orange
3    Green     2  Winter    Kiwi
4   Yellow     3  Winter   Lemon
5    Green     2  Winter    Kiwi
6   Yellow     3  Winter   Lemon
7    Green     1  Summer   Grape
8      Red     3  Spring   Apple
9      Red     3  Winter  Orange
10   Green     3  Winter    Kiwi
11  Yellow     2  Winter   Lemon
12     Red     1  Summer   Grape
13   Green     3  Spring   Apple
14     Red     3  Winter  Orange
15   Green     2  Winter    Kiwi
16  Yellow     2  Winter   Lemon
17     Red     1  Summer   Grape
18   Green     3  Spring   Apple
19     Red     3  Winter  Orange
20   Green     3  Winter    Kiwi
21  Yellow     3  Winter   Lemon
22     Red     1  Summer   Grape
23   Green     3  Spring   Apple
24     Red     3  Winter  Orange
25   Green     2  Winter    Kiwi
26  Yellow     3  Winter   Lemon
27     Red     1  Summer   Grape
28   Green     3  Spring   Apple
29     Red

### Definissez X de type np.array qui contiendra les valeurs des attributs ET des labels (pour faciliter le partinioning par la suite)

In [4]:
X = training_data.values

In [5]:
# Nous devons maintenant stocker les differents titres de categrories pour pouvoir afficher notre arbre
header = list(training_data.columns)

### Nous vous donnons la fonction permettant de trouver les differentes valeurs possibles pour chaque categorie, testez la pour mieux la comprendre 

In [6]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])
# La spécifité de l'objet set est qu'il ne peut pas contenir de doublons. Ses valeurs sont donc uniques.

### Testez votre fonction pour les differents attributs et labels ###

In [7]:
# Afficher les differentes saisons avec unique_vals
print(unique_vals(X, 2))

{'Summer', 'Spring', 'Winter'}


In [8]:
# Afficher les differentes couleurs avec unique_vals
print(unique_vals(X, 0))

{'Green', 'Yellow', 'Red'}


In [9]:
# Afficher les differents fruits avec unique_vals
print(unique_vals(X, 1))

{1, 2, 3}


### Definir la fonction permettant de compter le nombre de fruits présents par categorie *(nombre de citrons, nombre de pommes, etc.)* 
#### La valeur de retour doit être un dictionnaire

In [10]:
def class_counts(rows):
    counts = {v : 0 for v in unique_vals(rows, -1)}
    for row in rows:
        counts[row[-1]] += 1    
    return counts

In [11]:
## Lancez votre fonction pour votre dataset :
class_counts(X)
## Elle devrait retourner : {'Grape': 19, 'Apple': 19, 'Orange': 20, 'Kiwi': 22, 'Lemon': 20} 

{'Orange': 20, 'Lemon': 20, 'Apple': 19, 'Grape': 19, 'Kiwi': 22}

### Definition de la fonction is_numeric permettant de savoir si une valeur est de type int/float ou non. ###

In [12]:
def is_numeric(value):
    if type(value) == int or type(value) == float:
        return True
    else:
        return False

In [13]:
## Testez votre fonction pour une valeur numerique (int ou float) ...
print(is_numeric(4867))
print(is_numeric(42.4233333))
print(is_numeric(1/3))

True
True
True


In [14]:
#... et pour une valeur non-numérique 
print(is_numeric(X))
print(is_numeric(list()))
print(is_numeric('lol'))
print(is_numeric(True))
print(is_numeric('42'))

False
False
False
False
False


### Pour partitionner le dataset, nous allons utiliser une classe "Questions" ! Cette classe enregistre un numero de colonne (Ex : 0 pour la couleur) et une valeur de la colonne (ex : Vert).  
### La fonction match est utilisee pour comparer la valeur d'un exemple a celle de la question (voir commentaires fonction match).###
#### Vous avez ici directement le constructeur et la fonction d'affichage. A vous de trouver la fonction match####

In [15]:
class Question:
    
    def __init__(self, column, value):
        # Le Constructeur  permet d'attribuer les variables données à la classe
        self.column = column
        self.value = value

    def match(self, example):
        # Compare la valeur donnée pour la question (value), a la valeur correspondante d'example 
        # (example est une ligne)
        # ATTENTION : La valeur de retour doit etre un boolean (True / False)
        if is_numeric(self.value):
            return self.value <= example[self.column] 
        else:
            return self.value == example[self.column] 
        
    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

#### Testez vos questions

In [16]:
# Ecrivez une question pour un attribut numerique
q1 = Question(1, 2)

In [17]:
# Et une deuxieme pour un attribut catégorique (couleurs / saisons) : 
q2 = Question(2, 'Winter')

In [18]:
# Choisissez un exemple dans le dataset et vérifiez avec la fonction match s'il est de couleur rouge
example = X[np.random.randint(X.shape[0])]
print(example)
q3 = Question(0, 'Red')
print(q3.match(example))

['Yellow' 3 'Winter' 'Lemon']
False


In [19]:
# Pareil, mais tester pour size >= 2 (Attention à l'inegalité)
example = X[np.random.randint(X.shape[0])]
print(example)
q4 = Question(1, 2)
print(q4.match(example))

['Green' 2 'Winter' 'Kiwi']
True


### Vous devez definir la fonction partition qui prend en paramètres le dataset et une question, et les split en fonction de leur reponse (True / False) --> Ce seront nos 2 branches partant d'un noeud.

In [20]:
def partition(rows, question):
    # Pour chaque ligne dans le dataset, partionnez le en 2 en fonction du resultat de la question.
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)    
    return true_rows, false_rows

In [21]:
# Testons notre fonction partition pour isoler les lignes "rouges"
true_rows, false_rows = partition(X, q3)

In [22]:
# Et verifiez le contenue de vos 2 nodes
print(np.array(true_rows))
print(np.array(false_rows))

[['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 3 'Spring' 'Apple']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['Red' 3 'Winter' 'Orange']
 ['Red' 1 'Summer' 'Grape']
 ['

### Maintenant un peu de maths !
#### Nous allons calculer l'impurity d'une liste d'exemple grace a GINI 
#### ATTENTION : La fonction doit pouvoir prendre les listes qui suivent comme les lignes d'exemples contenues dans X (ex : gini(X[0]) ))

In [23]:
def gini(rows):
    impurity = 1
    labs = unique_vals(rows, 0)
    count = class_counts(rows)
    length = len(rows)
    for lab in labs:
        impurity -= (count[lab] / length)**2
    return impurity

In [24]:
# Pour verifier la fonction tester l'exemple suivant 
no_mixing = [['Apple'], ['Apple']]
# Que devrait-etre votre resultat ?
gini(no_mixing)

0.0

In [25]:
 # Maintenant avec un autre
some_mixing = [['Apple'], ['Orange'], ['Orange'], ['Apple']]
#  Que devriez vous obtenir comme resultat ?
gini(some_mixing)

0.5

In [26]:
# Et maintenant pour un dataset avec tous nos fruits
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Lemon'],
                  ['Kiwi']]
# Vous devriez obtenir environ 0.8
gini(lots_of_mixing)

0.7999999999999998

### Calculons l'information gain 

In [27]:
"""Information Gain :
The uncertainty of the starting node, minus the weighted impurity of
two child nodes.
"""

def info_gain(left, right, current_uncertainty):
    #left : true_rows
    #right : false_rows
    llength = len(left)
    rlength = len(right)
    p1 = llength / (llength + rlength)
    p2 = rlength / (llength + rlength)
    info_gain = current_uncertainty - ((p1 * gini(left)) + (p2 * gini(right)))    
    return info_gain

In [28]:
# Calculez l'uncertainy de notre dataset.
#print([[r[-1]] for r in X])

current_uncertainty = gini([[r[-1]] for r in X])
print(current_uncertainty)
# Vous devriez obtenir environ 0.8

0.7993999999999999


In [29]:
# Quel est notre information gain du Dataset si nous partitionnons avec la question couleur == rouge ??
info_gain1 = info_gain([[r[-1]] for r in true_rows], [[r[-1]] for r in false_rows], current_uncertainty)
print(info_gain1)
# Resultat : environ 0.1835

0.18349415720891116


In [30]:
# De même pour la question "Size >= 2"
true_rows2, false_rows2 = partition(X, q4)
info_gain2 = info_gain([[r[-1]] for r in true_rows2], [[r[-1]] for r in false_rows2], current_uncertainty)
print(info_gain2)
# Resultat : environ 0.1925

0.19248641975308634


### Pour ne pas tout tester manuellement, construisez la fonction vous permettant de trouver la question donnant le meilleur information gain en iterant sur chaque feature / value.

In [38]:
def find_best_split(rows):
    best_gain = 0
    best_question = None  # 1 Objet Question
    current_uncertainty = gini([[r[-1]] for r in rows])
    for i in range(rows.shape[1] - 1):
        u_vals = unique_vals(X, i)
        for row in rows:
            q = Question(i, row[i])
            tr, fr = partition(rows, q)
            tr = [[r[-1]] for r in tr]
            fr = [[r[-1]] for r in fr]
            ig = info_gain(tr, fr, current_uncertainty)
            if ig > best_gain:
                best_gain = ig
                best_question = q
    return best_gain, best_question

In [39]:
### Testez votre fonction sur le dataset
best_gain, best_question = find_best_split(X)
print(best_question)
# Vous devriez avoir : is color == yellow ?

Is Color == Yellow?


### Nous allons maintenant construire notre Decision Tree 
#### Les objets Leaf et Decision_Node sont donnés,  à vous de trouver la fonction qui permettra de construire l'arbre (build_tree)

In [40]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [41]:
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [45]:
def build_tree(rows):
    """Builds the tree.
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    bg, bq = find_best_split(rows)
    #print(bg)
    if bg > 0:
        #print(bq.column)
        tr, fr = partition(rows, bq)
        tr = np.array(tr)
        fr = np.array(fr)
        #np.delete(tr, bq.column, 0)
        #np.delete(fr, bq.column, 0)
        #print(tr)
        tb = build_tree(np.array(tr))
        fb = build_tree(np.array(fr))
        return (Decision_Node(bq, tb, fb))
    else:
        return (Leaf(rows))
    # Rappelez-vous de la condition d'arret et de la creation d'une leaf


### Fonction d'affichage de l'arbre (c'est cadeau ;) )

In [46]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

### Maintenant, creez votre arbre et imprimez le ! ;)

In [47]:
my_tree = build_tree(X)

In [48]:
print_tree(my_tree)

Is Color == Yellow?
--> True:
  Predict {'Lemon': 20}
--> False:
  Is Season == Winter?
  --> True:
    Is Color == Red?
    --> True:
      Predict {'Orange': 20}
    --> False:
      Predict {'Kiwi': 22}
  --> False:
    Is Size >= 3?
    --> True:
      Predict {'Apple': 19}
    --> False:
      Predict {'Grape': 19}


### Construisez la fonction classify qui permettra de parcourir l'abre pour un exemple donné. Elle nous retourne les predictions quand elle atteint une leaf (leaf.predictions)
#### Classify est une fonction recurssive allant de node en node :)
#### PS : man isinstance()

In [51]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [53]:
# Test de la fonction avec le 16eme data sample. Vous devriez obtenir {'Kiwi': 22}
classify(X[15], my_tree)

{'Kiwi': 22}

In [54]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [55]:
# Maintenant avec une fonction d'affichage un peu plus plaisante. Resultat : {'Grape': '100%'}
print_leaf(classify(X[0], my_tree))

{'Grape': '100%'}

In [56]:
# Tester avec ce dataset, puis essayer de changer les valeurs pour mieux comprendre le comportement de l'arbre
testing_data = [
    ['Green', 2, 'Winter','Kiwi'],
    ['Yellow', 3, 'Winter', 'Lemon'],
    ['Red', 1, 'Summer', 'Grape'],
    ['Red', 2, 'Summer', 'Grape'],
    ['Yellow', 3, 'Winter', 'Lemon'],
]

In [57]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[3], print_leaf(classify(row, my_tree))))

Actual: Kiwi. Predicted: {'Kiwi': '100%'}
Actual: Lemon. Predicted: {'Lemon': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Lemon': '100%'}
